In [38]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2
import warnings
# importing os module 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# Imports PIL module 
from keras.models import Model
from PIL import Image # for grabbing images
from itertools import chain #for target labels 
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from pathlib import Path
import seaborn as sns
import pandas as pd
import tensorflow
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
warnings.filterwarnings("ignore")
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from scipy.stats import multivariate_normal

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints` #It was automatically creating this file and causing problems

## Get Training and Test Samples as NumPy Arrays

In [ ]:
X_train=np.load("train_data.npy");
t_train=np.load("train_label.npy");
print(t_train.shape)
num_of_output_classes=len(np.unique(t_train))
X_train=tensorflow.keras.applications.vgg16.preprocess_input(X_train, data_format=None);

(3360,)


In [ ]:
def train_transfer_learning_CNN(num_of_classes,save_file_as,data_train, labels_train, img_size=214, epochs=5, learning_rate=1e-4):
    
    # load data and labels
    #data_train = np.load(data_train_path)
    #labels_train_temp = np.load(labels_train_path)
    
    # preprocessing
    #gray_scaled=Invert_gray_scale(data_train,300) #Converts all 300x300 images to grey scale
    
    #dilated_data=Dilation(gray_scaled,300) # Dilates all 300x300 images
    
    #bkg_removed_matrix=Remove_bkg_noise(dilated_data,300) #Removes background from 300x300 images
    
    #resized_matrix=resize(bkg_removed_matrix,img_size) #Resizes to img_size x img_size
 
    # Normalize data
    labels_train = np_utils.to_categorical(labels_train, num_of_classes)
    
    # specify a new input shape to replace VGG16 input layer for our data
    new_input = (img_size, img_size, 3)
    
    # load model
    model = VGG16(include_top=False, input_shape=new_input, pooling='max')
    
    #remove output layer to replace with new one to allow for classification of the 3 classes instead of the original VGG16
    # classification 
    flat1 = Flatten()(model.layers[-1].output) # remove output layer
    class1 = Dense(1024, activation='relu')(flat1) # new dense layer
    output = Dense(num_of_classes, activation='softmax')(class1) # for 3 classes
    
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    
    # print summary of model
    model.summary()

    # set Adam learning rate
    adam = Adam(lr=learning_rate)

    # We add metrics to get more results you want to see
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # fit model
    print('Training ------------')
    history = model.fit(data_train,labels_train, epochs=epochs)
                
    # plot training accuracy
    plt.plot(history.history['accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['Training'], loc='upper left')
    plt.show()

    # save trained VGG16 CNN model
    model.save(save_file_as)

## Training on all images

In [ ]:
# train VGG16 CNN if there is no file
if os.path.isfile('vgg16_trained_cnn.hdf5')==False:
    train_transfer_learning_CNN(num_of_output_classes,'vgg16_trained_cnn.hdf5',X_train,t_train, img_size=100, epochs=5, learning_rate=1e-4)

## Training performance when randomly pulling out 50% of the data

In [ ]:
if os.path.isfile('vgg16_rand_pullout_50.hdf5')==False:
    per_keep=.5 #keep 50% of data
    X_train_rand=X_train[0:int(per_keep*X_train.shape[0]),:,:,:]
    t_train_rand=t_train[0:int(per_keep*t_train.shape[0])]
    print("New Training shape")
    print(t_train_rand.shape)
    print(X_train_rand.shape)
    train_transfer_learning_CNN(num_of_output_classes,'vgg16_rand_pullout_50.hdf5',X_train_rand,t_train_rand, img_size=100, epochs=5, learning_rate=1e-4)

## Training performance when randomly pulling out 80% of the data

In [ ]:
if os.path.isfile('vgg16_rand_pullout_80.hdf5')==False:
    per_keep=.2 #keep 20% of data
    X_train_rand=X_train[0:int(per_keep*X_train.shape[0]),:,:,:]
    t_train_rand=t_train[0:int(per_keep*t_train.shape[0])]
    print("New Training shape")
    print(t_train_rand.shape)
    print(X_train_rand.shape)

    train_transfer_learning_CNN(num_of_output_classes,'vgg16_rand_pullout_80.hdf5',X_train_rand,t_train_rand, img_size=100, epochs=5, learning_rate=1e-4)